In [ ]:
'''
This is to prepare semantically segmented images for training.
Original ss images contain multiple colour labels for roads,
lane, markings, cars, people etc.

The purpose of this step is to create a single mask for each image.
E.g. all lane markings and roads to combine together and be the "True"
mask where any other colours would to to "False" or no mask

It depends on what you are trying to identify in the images,

Also trying simpler task with just one mask first
'''

In [2]:
import cv2
import numpy as np

In [7]:
#pick one image and show it
test_img = cv2.imread('C:/SelfDrive/out_sem/sem/1679033081006739600.png')
cv2.imshow('example',test_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
'''
looking for this colour rgb 128, 68, 128
red 128
green 68
blue 128

expecing cv2 to load image as B G R
so we expect 128 68 128 as 128 swapped around still givess the same value
'''

'\nlooking for this colour rgb 128, 68, 128\nred 128\ngreen 68\nblue 128\n\nexpecing cv2 to load image as B G R\nso we expect 128 68 128 as 128 swapped around still givess the same value\n'

In [14]:
#road colour mask
# first attempt - did not work - image came black - no colour picked up  untill i added dtype=... to upper and lower
lower_colour = np.array([128,64,128],dtype='uint8')
upper_colour = np.array([128,64,128],dtype='uint8')
road_mask = cv2.inRange(test_img,lower_colour,upper_colour)
cv2.imshow('road mask',road_mask)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [15]:
'''
lane markings
rgb 157, 234, 50  -> bgr 50,234,157
'''

lower_colour = np.array([50,234,157],dtype='uint8')
upper_colour = np.array([50,234,157],dtype='uint8')
lane_mask = cv2.inRange(test_img,lower_colour,upper_colour)
cv2.imshow('lanes mask',lane_mask)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [16]:
# now combine both masks

combined_mask = cv2.bitwise_or(road_mask,lane_mask)
cv2.imshow('Combined mask',combined_mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [20]:
# clean code for iterating over images
img_name = '1679033081006739600'
image = cv2.imread('C:/SelfDrive/out_sem/sem/'+img_name+'.png')
lower_colour = np.array([128,64,128],dtype='uint8')
upper_colour = np.array([128,64,128],dtype='uint8')
road_mask = cv2.inRange(image,lower_colour,upper_colour)
lower_colour = np.array([50,234,157],dtype='uint8')
upper_colour = np.array([50,234,157],dtype='uint8')
lane_mask = cv2.inRange(image,lower_colour,upper_colour)
combined_mask = cv2.bitwise_or(road_mask,lane_mask)
cv2.imwrite('C:/SelfDrive/out_sem/msk/'+img_name+'.png',combined_mask)

True

In [22]:
# generate all masks - 
# this is ROAD OR LANE MARKING

import os

in_path = 'C:/SelfDrive/out_sem/sem'
out_path = 'C:/SelfDrive/out_sem/msk'
images = [f.split('.')[0] for f in os.listdir(in_path) if f.endswith(".png")]

road_lower_colour = np.array([128,64,128],dtype='uint8')
road_upper_colour = np.array([128,64,128],dtype='uint8')

lane_lower_colour = np.array([50,234,157],dtype='uint8')
lane_upper_colour = np.array([50,234,157],dtype='uint8')


for example in images:
    img_path = in_path+'/'+example+'.png'
    image = cv2.imread(img_path)
    road_mask = cv2.inRange(image,road_lower_colour,road_upper_colour)
    lane_mask = cv2.inRange(image,lane_lower_colour,lane_upper_colour)
    combined_mask = cv2.bitwise_or(road_mask,lane_mask)
    cv2.imwrite(out_path+'/'+example+'.png',combined_mask)    

In [ ]:
'''need: for roads without edge marking, create edge marking
Loop through all pixels of the road colour rgb 128, 68, 128:

when a neighbour (right or left) color one of curbs, edges, etc
 RGB values
 180, 165, 180 1679213618266558700.png
 244, 35, 232 present in 1679213637770789100.png
 102, 102, 156 present in 1679213964177331800.png
 145, 170, 100 present in 1679213676655760900.png
 81, 0, 81 present in 1679213676655760900.png

Important -
 - and is not car colour and 
 - not people colour
 - not lane marking

Then change the neighbour's colour to a new dedicated colour
and change the neighbour+1 and + 2 and +3 colour too - in same direction as the neighbour

the outcome - a think edge to the right or left from the road
'''


In [36]:
# example - detect cases specified above
# next - make a function to draw thick edges - 10-15 pixel wide

import cv2
import numpy as np


in_path = 'C:/SelfDrive/out_sem/sem'
example = '1679213676655760900'
img_path = in_path+'/'+example+'.png'
image = cv2.imread(img_path)

road_colour = [[128, 64, 128],[128, 68, 128]]
colour_search = [[156,102,102],[180,165,180],[232,35,244],[100,170,145],[81, 0, 81],[100, 100, 150]] # these are BGR
replace_colour = [50,234,157]


def is_colour(pxl,clr_lst):
    result = False
    for clr in clr_lst:
        #print(clr, "current pixel:", pxl)
        if (pxl[0] == clr[0]) and (pxl[1] == clr[1]) and (pxl[2] == clr[2]):
            result = True
    return result


thickness=6  

for y in range(image.shape[0]):
    for x in range(image.shape[1]): #if this is a road pixel
        if is_colour(image[y,x],road_colour):
            # look left
            if x>2 and is_colour(image[y,x-3],colour_search):
                for pos in range(thickness):
                    new_x = x - (pos+1)
                    if new_x>= 0: 
                        image[y,new_x] = replace_colour
                    
            # Look right
            if x<image.shape[1]-4 and is_colour(image[y,x+3],colour_search):
                for pos in range(thickness):
                    new_x = x + (pos+1)
                    if new_x<image.shape[1]:
                        image[y,new_x] = replace_colour
                
                

cv2.imshow('Example',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [34]:
image.shape

(480, 640, 3)

In [38]:
# generate all additional lane marking

import os

in_path = 'C:/SelfDrive/out_sem/sem'
out_path = 'C:/SelfDrive/out_sem/sem_ln'
images = [f.split('.')[0] for f in os.listdir(in_path) if f.endswith(".png")]

road_colour = [[128, 64, 128],[128, 68, 128]]
colour_search = [[156,102,102],[180,165,180],[232,35,244],[100,170,145],[81, 0, 81],[100, 100, 150]] # these are BGR
replace_colour = [50,234,157]

thickness=6  

def is_colour(pxl,clr_lst):
    result = False
    for clr in clr_lst:
        #print(clr, "current pixel:", pxl)
        if (pxl[0] == clr[0]) and (pxl[1] == clr[1]) and (pxl[2] == clr[2]):
            result = True
    return result


for example in images:
    img_path = in_path+'/'+example+'.png'
    image = cv2.imread(img_path)

    for y in range(image.shape[0]):
        for x in range(image.shape[1]): #if this is a road pixel
            if is_colour(image[y,x],road_colour):
                # look left
                if x>2 and is_colour(image[y,x-3],colour_search):
                    for pos in range(thickness):
                        new_x = x - (pos+1)
                        if new_x>= 0: 
                            image[y,new_x] = replace_colour

                # Look right
                if x<image.shape[1]-4 and is_colour(image[y,x+3],colour_search):
                    for pos in range(thickness):
                        new_x = x + (pos+1)
                        if new_x<image.shape[1]:
                            image[y,new_x] = replace_colour
    
    
    cv2.imwrite(out_path+'/'+example+'.png',image)    

In [1]:
# generate all masks - 
# this is just LANE MARKING masks

import os
import cv2
import numpy as np

in_path = 'C:/SelfDrive/out_sem/sem_ln'
out_path = 'C:/SelfDrive/out_sem/msk_ln'
images = [f.split('.')[0] for f in os.listdir(in_path) if f.endswith(".png")]


lane_lower_colour = np.array([50,234,157],dtype='uint8')
lane_upper_colour = np.array([50,234,157],dtype='uint8')


for example in images:
    img_path = in_path+'/'+example+'.png'
    image = cv2.imread(img_path)
    lane_mask = cv2.inRange(image,lane_lower_colour,lane_upper_colour)
    cv2.imwrite(out_path+'/'+example+'.png',lane_mask)    